In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# API Keys from environment variables
SHIPDAY_API_KEY = os.getenv('SHIPDAY_API_KEY')
OPENROUTE_API_KEY = os.getenv('OPENROUTE_API_KEY')
GOOGLE_MAPS_API_KEY = os.getenv('GOOGLE_MAPS_API_KEY')

# Validate API keys are present
if not all([SHIPDAY_API_KEY, OPENROUTE_API_KEY, GOOGLE_MAPS_API_KEY]):
    raise ValueError("Missing required API keys in environment variables")

In [ ]:
import folium
from geopy.distance import geodesic
from IPython.display import display
import requests
import time
from shipday import Shipday
from shipday.order import Address
import pandas as pd
import math
import json 
import googlemaps
import os
from datetime import datetime
import time
from functools import wraps

# Starting point (latitude, longitude)
start_lat, start_lon = 32.0646381, -81.2050952

# Function to validate coordinates
def validate_coordinates(lat, lon):
    """Validate latitude and longitude values"""
    if not (-90 <= lat <= 90) or not (-180 <= lon <= 180):
        raise ValueError(f"Invalid coordinates: lat={lat}, lon={lon}")
    return lat, lon

# Data Path handling
def get_project_root():
    return os.path.abspath(os.path.dirname(os.path.dirname(__file__)))

def get_data_path(subfolder):
    data_path = os.path.join(get_project_root(), 'data', subfolder)
    os.makedirs(data_path, exist_ok=True)
    return data_path

output_dir = get_data_path('JSONs')
maps_data_dir = get_data_path('Maps_data')
maps_html_dir = get_data_path('Maps_HTML')

shipday_obj = Shipday(api_key=SHIPDAY_API_KEY) # Initialize Google Maps client

# Function to generate points 10 miles away from the starting point
def generate_points(start_lat, start_lon, num_points=16, miles=15):
    start_lat, start_lon = validate_coordinates(start_lat, start_lon)
    if not (1 <= num_points <= 20):
        raise ValueError("num_points must be between 1 and 20")
    if not (1 <= miles <= 50):
        raise ValueError("miles must be between 1 and 50")
    points = []
    for i in range(num_points):
        angle = i * (360 / num_points)
        destination = geodesic(miles).destination((start_lat, start_lon), angle)
        points.append((destination.latitude, destination.longitude))
    return points

# Generate points
points = generate_points(start_lat, start_lon)
# Initialize API call counter
api_call_counter = 0

def rate_limit(calls_per_second=1):
    min_interval = 1.0 / calls_per_second
    last_call_time = {}
    
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            current_time = time.time()
            if func in last_call_time:
                elapsed = current_time - last_call_time[func]
                if elapsed < min_interval:
                    time.sleep(min_interval - elapsed)
            result = func(*args, **kwargs)
            last_call_time[func] = time.time()
            return result
        return wrapper
    return decorator

@rate_limit(calls_per_second=0.5)  # Limit to 1 call every 2 seconds

# Function to calculate driving distance using OpenRouteService API
def calculate_driving_distance(start_lat, start_lon, points):
    distances = []
    api_key = OPENROUTE_API_KEY
    url = 'https://api.openrouteservice.org/v2/directions/driving-car'
    #print(f"API call timestamp: {time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime())}")

    for point in points:
        coords = [[start_lon, start_lat], [point[1], point[0]]]
        headers = {'Authorization': api_key, 'Content-Type': 'application/json'}
        body = {'coordinates': coords}
        global api_call_counter
        api_call_counter += 1
        print(f"API call count: {api_call_counter}")
        # Capture the current time before the API call
        start_time = time.time()
        
        response = requests.post(url, json=body, headers=headers)
        
        # Capture the current time after the API call
        end_time = time.time()
        
        #print(f"Response timestamp: {time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime())}")
        #print(f"API call to {url} took {end_time - start_time:.2f} seconds")
        
        if response.status_code == 200:
            data = response.json()
            distance = data['routes'][0]['summary']['distance'] / 1609.34  # Convert to miles
            distances.append(distance)
        else:
            distances.append(None)
        
    return distances

# Calculate driving distances
distances = calculate_driving_distance(start_lat, start_lon, points)

# Create a matrix of points (latitude and longitude) and distances before filtering out valid points
points_distances_matrix = list(zip(points, distances))

# Display the matrix
for point, distance in points_distances_matrix:
    print(f"First display Point: {point}, Distance: {distance}")

points_save=points

API call count: 1
API call count: 2
API call count: 3
API call count: 4
API call count: 5
API call count: 6
API call count: 7
API call count: 8
API call count: 9
API call count: 10
API call count: 11
API call count: 12
API call count: 13
API call count: 14
API call count: 15
API call count: 16
First display Point: (32.19990835055135, -81.2050952), Distance: 14.394534405408429
First display Point: (32.189597002295194, -81.14422183697542), Distance: None
First display Point: (32.16023904061263, -81.09265191903765), Distance: None
First display Point: (32.11631906599021, -81.05825129890849), Distance: None
First display Point: (32.064538549667574, -81.04624209181833), Distance: 10.904656567288454
First display Point: (32.012786767415136, -81.05841662679751), Distance: 12.341083922601813
First display Point: (31.968936163213826, -81.09288572879652), Distance: 14.911205835932744
First display Point: (31.939647572290703, -81.14438716601842), Distance: 16.134191656206895
First display Point: 

# SHIPDAY API AVAILABILITY PYTHON DOC:
from shipday import Shipday
from shipday.order import Address

API_KEY = 'SHIPDAY_API_KEYshipday_obj = Shipday(api_key=API_KEY)

pickup = Address(street='890 Geneva Av', city='San Fransisco', state='California', zip='CA 94132',country='USA')
delivery = Address(street='556 Crestlake Dr', city='San Francisco', state='California', country='USA')

shipday_obj.OnDemandDeliveryService.check_availability(pickup_address=pickup, delivery_address=delivery)

In [ ]:
# Initialize Google Maps client
gmaps = googlemaps.Client(key=GOOGLE_MAPS_API_KEY)

# Function to get addresses from coordinates using OpenRouteService API
def get_address_from_coordinates(lat, lon):
    api_key = OPENROUTE_API_KEY
    url = 'https://api.openrouteservice.org/geocode/reverse'
    params = {
        'api_key': api_key,
        'point.lat': lat,
        'point.lon': lon,
        'size': 1
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        if data['features']:
            address = data['features'][0]['properties']['label']
            return address
    return None

# Function to format and validate address using Google Maps Geocoding API
def format_address(address):
    geocode_result = gmaps.geocode(address)
    if geocode_result:
        address_components = geocode_result[0]['address_components']
        address_dict = {component['types'][0]: component['long_name'] for component in address_components}
        street = address_dict.get('route', '') + ' ' + address_dict.get('street_number', '')
        city = address_dict.get('locality', '')
        state = address_dict.get('administrative_area_level_1', '')
        zip_code = address_dict.get('postal_code', '')
        country = address_dict.get('country', '')
        return {
            'street': street,
            'city': city,
            'state': state,
            'zip': zip_code,
            'country': country
        }
    return None

# Function to recalculate distances for points that were moved closer to the starting point
def recalculate_distances(start_lat, start_lon, points):
    return calculate_driving_distance(start_lat, start_lon, points)

# Function to move points closer to the starting point
def move_closer(start, end, factor):
    # Calculate the new coordinates by scaling the difference
    print(f"Moving point {end} closer to {start} by a factor of {factor}")
    new_lat = start[0] + (end[0] - start[0]) * factor
    new_lon = start[1] + (end[1] - start[1]) * factor
    print(f"New coordinates: ({new_lat}, {new_lon})")
    return (new_lat, new_lon)

# Map pickup address
#pickup_address = get_address_from_coordinates(start_lat, start_lon)                                            # Use this line to get the address from coordinates
pickup_address='1450 Dean Forest Road, Garden City, GA 31405-9365, USA'                                              # Use this line to manually set the address
print(f"Pickup address: {pickup_address}")

# Format and validate pickup address
formatted_pickup_address = format_address(pickup_address)
print(f"Formatted pickup address: {formatted_pickup_address}")

# Map Pickup address with Starting point for Shipday API
pickup = Address(
    street=formatted_pickup_address['street'],
    city=formatted_pickup_address['city'],
    state=formatted_pickup_address['state'],
    zip=formatted_pickup_address['zip'],
    country=formatted_pickup_address['country']
)

def rescale_points(start_lat, start_lon, points, distances, valid_response_names, max_cycles=12):
    factor = 0.95  # 10% shrinkage in distance = 0.9 factor
    all_points = []
    all_distances = []
    all_responses = []
    all_data = pd.DataFrame(columns=['Point#', 'Formatted Address', 'Street', 'City', 'State', 'Zip', 'Country', 'Latitude', 'Longitude'])
    cycle_count = 0
    
    # Track points that have already been validated
    validated_points = set()
    
    # Create a dictionary to track points that need to be moved
    points_to_move = {point: point for point in points}
    
    while cycle_count < max_cycles:
        print(f"Cycle {cycle_count + 1}")
        new_points = {}
        valid_points = []
        valid_distances = []
        
        for point in points:
            if point in validated_points:
                continue
            
            current_point = points_to_move.get(point, point)
            
            # Get and validate address
            address = get_address_from_coordinates(current_point[0], current_point[1])
            if not address:
                print(f"No address found for point {current_point}, moving closer.")
                new_point = move_closer((start_lat, start_lon), current_point, factor)
                new_points[point] = new_point
                continue

            # Format address
            formatted_address = format_address(address)
            if not formatted_address:
                print(f"Invalid address format for {address}, moving closer.")
                new_point = move_closer((start_lat, start_lon), current_point, factor)
                new_points[point] = new_point
                continue

            # Check availability
            delivery_address = Address(
                street=formatted_address['street'],
                city=formatted_address['city'],
                state=formatted_address['state'],
                zip=formatted_address['zip'],
                country=formatted_address['country']
            )
            
            response = shipday_obj.OnDemandDeliveryService.check_availability(
                pickup_address=pickup, 
                delivery_address=delivery_address
            )

            # Validate response
            if isinstance(response, str):
                try:
                    response_data = json.loads(response)
                except json.JSONDecodeError:
                    print(f"Invalid JSON response for point {current_point}, moving closer.")
                    new_point = move_closer((start_lat, start_lon), current_point, factor)
                    new_points[point] = new_point
                    continue
            elif isinstance(response, list):
                response_data = response
            else:
                print(f"Unexpected response format for point {current_point}, moving closer.")
                new_point = move_closer((start_lat, start_lon), current_point, factor)
                new_points[point] = new_point
                continue

            # Save response data to file
            os.makedirs(output_dir, exist_ok=True)
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            filename = f'response_{timestamp}.json'
            file_path = os.path.join(output_dir, filename)
            with open(file_path, 'w') as file:
                json.dump(response_data, file)

            # Check if any valid delivery service is available
            valid_service = False
            for item in response_data:
                if item.get('name') in valid_response_names and item.get('fee') is not None and not item.get('error'):
                    valid_service = True
                    valid_points.append(point)
                    valid_distances.append(distances[points.index(point)])
                    all_responses.append(response_data)
                    validated_points.add(point)

                    # Store the point data
                    point_data = {
                        'Point#': len(all_points) + 1,
                        'Formatted Address': formatted_address,
                        'Street': formatted_address['street'],
                        'City': formatted_address['city'],
                        'State': formatted_address['state'],
                        'Zip': formatted_address['zip'],
                        'Country': formatted_address['country'],
                        'Latitude': current_point[0],
                        'Longitude': current_point[1]
                    }
                    all_data = pd.concat([all_data, pd.DataFrame([point_data])], ignore_index=True)
                    print(f"Valid point found. all_data size: {all_data.shape[0]}")
                    break

            if not valid_service:
                print(f"No valid delivery service for point {current_point}, moving closer.")
                new_point = move_closer((start_lat, start_lon), current_point, factor)
                new_points[point] = new_point

        all_points.extend(valid_points)
        all_distances.extend(valid_distances)
        
        # Update points_to_move with new positions
        points_to_move.update(new_points)
        
        print(f"Cycle {cycle_count + 1} complete:")
        print(f"Total points: {len(points)}")
        print(f"Points to move: {len(new_points)}")
        print(f"Validated points: {len(validated_points)}/{len(points)}")
        
        if len(validated_points) == len(points):
            print("All points have been validated successfully.")
            break
        
        if not new_points:
            print("No new points to process, ending cycle.")
            break
        
        # Update distances for points that were moved
        current_points = [points_to_move[p] if p in points_to_move else p for p in points]
        distances = recalculate_distances(start_lat, start_lon, current_points)
        
        cycle_count += 1
    
    # Get final positions of all points
    final_points = [points_to_move[p] if p in points_to_move else p for p in points]
    print(f"Final points: {final_points}")
    return all_data, all_points, final_points, distances, all_responses

# Specify the valid response names
# Complete list: 'Dlivrd', 'DoorDash', 'DoorDash Catering', 'Uber' 
valid_response_names = ['DoorDash']

# Rescale points and recalculate distances
all_data, all_points, final_points, final_distances, all_responses = rescale_points(start_lat, start_lon, points, distances, valid_response_names)

Pickup address: 1450 Dean Forest Road, Garden City, GA 31405-9365, USA
Formatted pickup address: {'street': 'Dean Forest Road 1450', 'city': 'Garden City', 'state': 'Georgia', 'zip': '31405', 'country': 'United States'}
Cycle 1
No valid delivery service for point (32.19990835055135, -81.2050952), moving closer.
Moving point (32.19990835055135, -81.2050952) closer to (32.0646381, -81.2050952) by a factor of 0.95
New coordinates: (32.19314483802378, -81.2050952)
No valid delivery service for point (32.189597002295194, -81.14422183697542), moving closer.
Moving point (32.189597002295194, -81.14422183697542) closer to (32.0646381, -81.2050952) by a factor of 0.95
New coordinates: (32.183349057180436, -81.14726550512665)
No valid delivery service for point (32.16023904061263, -81.09265191903765), moving closer.
Moving point (32.16023904061263, -81.09265191903765) closer to (32.0646381, -81.2050952) by a factor of 0.95
New coordinates: (32.155458993582, -81.09827408308577)
No valid delivery 

C:\Users\JulianRamirezArboled\AppData\Local\Temp\ipykernel_1588\3819819461.py:182: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_data = pd.concat([all_data, pd.DataFrame([point_data])], ignore_index=True)


Valid point found. all_data size: 1
Cycle 3 complete:
Total points: 16
Points to move: 15
Validated points: 1/16
API call count: 49
API call count: 50
API call count: 51
API call count: 52
API call count: 53
API call count: 54
API call count: 55
API call count: 56
API call count: 57
API call count: 58
API call count: 59
API call count: 60
API call count: 61
API call count: 62
API call count: 63
API call count: 64
Cycle 4
No valid delivery service for point (32.18061543106646, -81.2050952), moving closer.
Moving point (32.18061543106646, -81.2050952) closer to (32.0646381, -81.2050952) by a factor of 0.95
New coordinates: (32.17481656451314, -81.2050952)
No valid delivery service for point (32.17177473885534, -81.15290390037681), moving closer.
Moving point (32.17177473885534, -81.15290390037681) closer to (32.0646381, -81.2050952) by a factor of 0.95
New coordinates: (32.16641790691257, -81.15551346535797)
No valid delivery service for point (32.14660395645775, -81.1086891419849), movi

In [ ]:
# Function to calculate distance between two points
def calculate_distance(lat1, lon1, lat2, lon2):
    return geodesic((lat1, lon1), (lat2, lon2)).miles

# Add a new column for distances
all_data['Distance'] = all_data.apply(
    lambda row: calculate_distance(start_lat, start_lon, row['Latitude'], row['Longitude']), axis=1
)

# Generate filename using pickup_address and current timestamp
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
pickup_address_clean = pickup_address.replace(' ', '_').replace(',', '').replace('-', '')
filename = f'{pickup_address_clean}_{timestamp}.csv'

# Save the DataFrame to CSV
all_data.to_csv(os.path.join(maps_data_dir, filename), index=False)
print(all_data)

   Point#                                  Formatted Address  \
0       1  {'street': 'Highlands Boulevard ', 'city': 'Sa...   
1       2  {'street': 'Berrien Street ', 'city': 'Savanna...   
2       2  {'street': 'De Renne Court ', 'city': 'Savanna...   
3       2  {'street': 'Chesterfield Court ', 'city': 'Sav...   
4       2  {'street': 'Bloomingdale Road ', 'city': 'Bloo...   
5       2  {'street': 'Adams Road ', 'city': 'Bloomingdal...   
6       7  {'street': 'North Coastal Highway ', 'city': '...   
7       7  {'street': 'Rivers End Drive ', 'city': 'Savan...   
8       7  {'street': 'Guana Lane ', 'city': 'Savannah', ...   
9      10  {'street': 'Rainbow Lane ', 'city': 'Georgetow...   
10     10  {'street': 'Hill Road ', 'city': 'Georgetown',...   
11     12  {'street': 'Tar Creek Road ', 'city': 'Savanna...   
12     13  {'street': 'Crossroads Parkway 125', 'city': '...   
13     13  {'street': 'Pinkney Road ', 'city': 'Savannah'...   
14     15  {'street': 'North Lathrop Ave

In [ ]:
# Function to calculate the angle of a point relative to the center
def calculate_angle(center, point):
    angle = math.atan2(point[1] - center[1], point[0] - center[0])
    return angle

# Function to create a map with the points
def create_map(start_lat, start_lon, all_data, final_points):
    center = (start_lat, start_lon)
    
    # Extract latitude and longitude from all_data
    all_points = all_data[['Latitude', 'Longitude']].values.tolist()
    
    # Calculate angles for each point and sort by angle
    points_with_angles = [(point, calculate_angle(center, point)) for point in all_points]
    points_with_angles.sort(key=lambda x: x[1])
    sorted_points = [point for point, angle in points_with_angles]
    
    m = folium.Map(location=[start_lat, start_lon], zoom_start=12)
    folium.Marker([start_lat, start_lon], popup='Start Point').add_to(m)
    
    # Add markers for each point in all_points (blue)
    for point in sorted_points:
        folium.Marker(point, popup=f'Point {sorted_points.index(point) + 1}', icon=folium.Icon(color='blue')).add_to(m)
    
    # Add markers for each point in final_points (red)
    for point in final_points:
        folium.Marker(point, popup=f'Final Point {final_points.index(point) + 1}', icon=folium.Icon(color='red')).add_to(m)
    
    # Create a polygon with the outline of the points in all_points
    folium.Polygon(locations=sorted_points, color="blue", weight=2.5, opacity=1).add_to(m)
    
    return m

# Create map
m = create_map(start_lat, start_lon, all_data, final_points)

# Display the map
display(m)

In [ ]:
# Save the map as an HTML file
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
pickup_address_clean = pickup_address.replace(' ', '_').replace(',', '').replace('-', '')
map_filename = os.path.join(maps_html_dir, f'map_{pickup_address_clean}_{timestamp}.html')
m.save(map_filename)
print(f"Map saved as {map_filename}")

Map saved as C:\Users\JulianRamirezArboled\Documents\Personal\ShipdayMappingProject\Maps_HTML\map_1450_Dean_Forest_Road_Garden_City_GA_314059365_USA_20250414_201841.html
